# Data Notes

<pre>
VARIABLE DESCRIPTIONS:
InvoiceNo:发票号码
StockCode:股票代码
Description:描述
Quantity:数量
InvoiceDate:发票日期
UnitPrice:单价
CustomerID:顾客ID
Country:国家
</pre>

___

In [1]:
import warnings
warnings.filterwarnings("ignore")

# Read Data

In [2]:
import pandas as pd
from pandas import Series,DataFrame

In [3]:
data = pd.read_csv('./data/E-commerce-data.csv')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
#data['InvoiceDate']

---

# Check Data

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null object
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


- **CustomerID** column has Null values

In [7]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


---

# Clean Data

In [9]:
data.shape

(541909, 8)

In [10]:
data_copy = data.copy()

In [11]:
data_copy = data_copy.dropna()
data_copy.shape

(406829, 8)

---

# Analysis Methods

- **Popular Items Analysis**
- **Cluster by Customer**
- **Customer Cluster Analysis**
- **Time Series Analysis**

---

# Popular Items

### Appeared in Invoice

In [12]:
# TOP 20
hot_items = data['Description'].value_counts()[:10]
hot_items

WHITE HANGING HEART T-LIGHT HOLDER    2369
REGENCY CAKESTAND 3 TIER              2200
JUMBO BAG RED RETROSPOT               2159
PARTY BUNTING                         1727
LUNCH BAG RED RETROSPOT               1638
ASSORTED COLOUR BIRD ORNAMENT         1501
SET OF 3 CAKE TINS PANTRY DESIGN      1473
PACK OF 72 RETROSPOT CAKE CASES       1385
LUNCH BAG  BLACK SKULL.               1350
NATURAL SLATE HEART CHALKBOARD        1280
Name: Description, dtype: int64

In [13]:
from pyecharts import Bar


attr = hot_items.index
v1 = hot_items.values
bar = Bar("Popular-items",height=400,width=700)
bar.add("", attr, v1, is_stack=True,mark_line=["max"],is_label_show=True,is_more_utils=True,
       xaxis_interval=0, xaxis_rotate=30, yaxis_rotate=30)
bar.render('./output/Popular-items.html')
bar

---

### Quantity Counts

In [14]:
grouped_items = data['Quantity'].groupby(data['Description'])
pd_item_quant = DataFrame(grouped_items.sum())

In [15]:
# TOP 10
top_quantity = pd_item_quant.sort_values('Quantity',ascending=False)[:10]
top_quantity

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53847
JUMBO BAG RED RETROSPOT,47363
ASSORTED COLOUR BIRD ORNAMENT,36381
POPCORN HOLDER,36334
PACK OF 72 RETROSPOT CAKE CASES,36039
WHITE HANGING HEART T-LIGHT HOLDER,35317
RABBIT NIGHT LIGHT,30680
MINI PAINT SET VINTAGE,26437
PACK OF 12 LONDON TISSUES,26315


In [16]:
#top_quantity.index

In [17]:
from pyecharts import Bar


items_index = top_quantity.index
items_value = top_quantity.Quantity.values
bar = Bar("Top-Quantity-Items",height=400,width=700)
bar.add("", items_index, items_value, is_stack=True,mark_line=["max"],is_label_show=True,is_more_utils=True,
       xaxis_interval=0, xaxis_rotate=30, yaxis_rotate=30)
bar.render('./output/Top-Quantity-Items.html')
bar

---

# Customer Analysis

In [18]:
# UnitPrice*Quantity
totalprice = data['UnitPrice']*data['Quantity']

In [19]:
customer_data = data.copy()

In [20]:
# insert into data
customer_data.insert(6,'totalprice',totalprice)

In [21]:
customer_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,totalprice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,15.30,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,20.34,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,22.00,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,20.34,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,20.34,17850.0,United Kingdom


## drop Null

In [22]:
customer_data = customer_data.dropna()

### Groupby CustomerID to Discover total price

In [23]:
grouped_totalamount = customer_data['totalprice'].groupby(customer_data['CustomerID'])
pd_totalamount = DataFrame(grouped_totalamount.sum())

In [24]:
pd_totalamount.head()

,totalprice
CustomerID,
12346.0,0.00
12347.0,4310.00
12348.0,1797.24
12349.0,1757.55
12350.0,334.40


### Groupby CustomerID to Discover frequency/sum item's category

In [25]:
grouped_frequency = customer_data['InvoiceNo'].groupby(customer_data['CustomerID'])
pd_frequency = DataFrame(grouped_frequency.count())

In [26]:
pd_frequency.head()

,InvoiceNo
CustomerID,
12346.0,2
12347.0,182
12348.0,31
12349.0,73
12350.0,17


---

In [27]:
pd_customer = pd.concat([pd_frequency,pd_totalamount],axis=1)
pd_customer.columns=['ItemsCount','TotalPrice']
pd_customer.head()

,ItemsCount,TotalPrice
CustomerID,,
12346.0,2,0.00
12347.0,182,4310.00
12348.0,31,1797.24
12349.0,73,1757.55
12350.0,17,334.40


---

# K-Means Cluster

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer

num_pipeline = Pipeline([
        #('imputer', Imputer(strategy="median")),
        ('std_scaler', StandardScaler())
    ])

customer_prepared = num_pipeline.fit_transform(pd_customer)

In [29]:
customer_prepared

array([[-0.3917197 , -0.23100099],
       [ 0.38265697,  0.29343167],
       [-0.26695902, -0.01231622],
       ...,
       [-0.34439668, -0.20951263],
       [ 2.85205812,  0.02390005],
       [-0.0991774 , -0.00744423]])

In [30]:
from sklearn.cluster import KMeans # 导入Kmeans
k=7 # 聚类类别数
kmodel = KMeans(n_clusters=k, n_jobs=4)
kmodel.fit(customer_prepared)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=7, n_init=10, n_jobs=4, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [31]:
kmodel.cluster_centers_

array([[-0.20408174, -0.13226353],
       [25.04225176,  7.65089359],
       [ 6.24067306,  9.7649091 ],
       [ 3.71391067, 29.11003993],
       [ 0.751671  ,  0.32488719],
       [ 0.84663765,  4.53215878],
       [ 3.4142294 ,  0.78153833]])

In [32]:
centre = num_pipeline.inverse_transform(kmodel.cluster_centers_)

In [33]:
c = pd.DataFrame(centre).sort_values(1)
c.columns=['ItemsCount','TotalPrice']
c.index = ['Ordinary','Normal Develop','Normal Maintain','High Develop','High Maintain','High Value','Top Value']

In [34]:
c

,ItemsCount,TotalPrice
Ordinary,45.615509,811.464393
Normal Develop,267.775475,4568.514214
Normal Maintain,886.673913,8321.458261
High Develop,289.850000,39145.576500
High Maintain,5914.000000,64776.602500
High Value,1543.666667,82150.438333
Top Value,956.333333,241136.560000


In [35]:
kmodel.labels_

array([0, 4, 0, ..., 0, 6, 0])

In [36]:
# make a copy
customer = pd_customer.copy()

In [37]:
centre_index = pd.DataFrame(centre).sort_values(1).index

centre_map = {
    centre_index[0]:'Ordinary',
    centre_index[1]:'Normal Develop',
    centre_index[2]:'Normal Maintain',
    centre_index[3]:'High Develop',
    centre_index[4]:'High Maintain',
    centre_index[5]:'High Value',
    centre_index[6]:'Top Value',
}

customer.insert(2,'Category',kmodel.labels_)
customer['Category'] = customer['Category'].map(centre_map)

In [38]:
customer.head()

,ItemsCount,TotalPrice,Category
CustomerID,,,
12346.0,2,0.00,Ordinary
12347.0,182,4310.00,Normal Develop
12348.0,31,1797.24,Ordinary
12349.0,73,1757.55,Ordinary
12350.0,17,334.40,Ordinary


---

## Analysis Cluster Result

### Customer Category Counts

In [39]:
customer['Category'].value_counts()

Ordinary           3707
Normal Develop      586
Normal Maintain      46
High Develop         20
High Value            6
High Maintain         4
Top Value             3
Name: Category, dtype: int64

**Divided into 7 Categories**

---

# TimeSeries Analysis

### Extract Date & Time

In [40]:
TimeSeries_data = customer_data[['InvoiceDate','totalprice']]
TimeSeries_data.head()

,InvoiceDate,totalprice
0,12/1/2010 8:26,15.30
1,12/1/2010 8:26,20.34
2,12/1/2010 8:26,22.00
3,12/1/2010 8:26,20.34
4,12/1/2010 8:26,20.34


In [41]:
import re

pattern_month = '[0-9]+'
pattern_time = '[0-9]+:[0-9]+'
pattern_hour = '[0-9]+'

In [42]:
monthlist = []
timelist = []
hourlist = []
for item in TimeSeries_data['InvoiceDate']:
    month = re.findall(pattern_month,item)[0]
    time = re.findall(pattern_time,item)[0]
    hour = re.findall(pattern_hour,time)[0]
    monthlist.append(month)
    timelist.append(time)
    hourlist.append(str(hour))

In [43]:
#monthlist

In [44]:
TimeSeries_data.insert(1,'Month',Series(monthlist))
TimeSeries_data.insert(2,'Time',Series(timelist))
TimeSeries_data.insert(3,'Hour',Series(hourlist))

In [45]:
TimeSeries_data.head()

,InvoiceDate,Month,Time,Hour,totalprice
0,12/1/2010 8:26,12,8:26,8,15.30
1,12/1/2010 8:26,12,8:26,8,20.34
2,12/1/2010 8:26,12,8:26,8,22.00
3,12/1/2010 8:26,12,8:26,8,20.34
4,12/1/2010 8:26,12,8:26,8,20.34


In [46]:
TimeSeries_data['Hour'] = pd.to_numeric(TimeSeries_data['Hour'])
TimeSeries_data['Month'] = pd.to_numeric(TimeSeries_data['Month'])

---

### Analysis Hot Date/Hour

In [47]:
grouped_date = TimeSeries_data['totalprice'].groupby(TimeSeries_data['Month'])
month_consume = DataFrame(grouped_date.count())
month_consume.head()

,totalprice
Month,
1.0,13141
2.0,11363
3.0,20969
4.0,17789
5.0,21759


In [48]:
grouped_hour = TimeSeries_data['totalprice'].groupby(TimeSeries_data['Hour'])
hour_consume = DataFrame(grouped_hour.count()).sort_index()
hour_consume.head()

,totalprice
Hour,
7.0,280
8.0,6311
9.0,16438
10.0,29518
11.0,37209


In [49]:
from pyecharts import Bar

hour_index =list(map(int, hour_consume.index))
hour_value =  hour_consume['totalprice'].values
bar = Bar("Hour-Consumption-Analysis")
bar.add("", hour_index, hour_value,mark_line=["average"],is_label_show=True,is_smooth=True,is_more_utils=True)

bar.render('./output/rush-hour-bar.html')
bar

---

In [50]:
from pyecharts import Bar

month_index =list(map(int, month_consume.index))
month_value =  month_consume['totalprice'].values
bar = Bar("Month-Consumption-Analysis")
bar.add("", month_index, month_value,mark_line=["average"],is_more_utils=True)

bar.render('./output/rush-month-consume.html')
bar

---